# HW1

## Personal info

Make sure you use your own student id

In [13]:
STUDENT_ID = "313551121"  # Replace with your actual student ID

print(f"Student ID: {STUDENT_ID}")

Student ID: 313551121


## Mount Google Drive
Mount Google Drive to access the dataset

Colab cannot directly access "Shared with me".

So you need to add a shortcut to your own Drive:

Open the shared link in Google Drive.

Right-click → “Add shortcut to Drive”.

Choose MyDrive → confirm.

Now the folder will appear under:

/content/drive/MyDrive/flattendata

In [14]:
# Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')

print("Google Drive mounted successfully!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully!


In [15]:
# Load and explore the dataset
import os
import torch
from torchvision import datasets, transforms

# Define data paths
TRAIN_DIR = "/content/drive/MyDrive/flattendata/train"
VALID_DIR = "/content/drive/MyDrive/flattendata/valid"
TEST_DIR = "/content/drive/MyDrive/flattendata/test"
TEST_CSV = "/content/drive/MyDrive/flattendata/test.csv"

# Basic transform for initial exploration
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Load datasets
print("Loading datasets...")
train_dataset = datasets.ImageFolder(root=TRAIN_DIR, transform=transform)
valid_dataset = datasets.ImageFolder(root=VALID_DIR, transform=transform)

# Display dataset information
print(f"Train samples: {len(train_dataset)}")
print(f"Valid samples: {len(valid_dataset)}")
print(f"Number of classes: {len(train_dataset.classes)}")

Loading datasets...
Train samples: 13492
Valid samples: 500
Number of classes: 100


## Data Handling
Below is the dataset class structure.
It defines a PyTorch Dataset that:
- Reads images from the provided file paths
- Converts them to grayscale, RGB, or another mode (depending on your choice)
- Applies optional transforms
- Returns (image, label) pairs for training, or just the image for inference


In [16]:
# Define the ClassificationDataset class here
from torch.utils.data import Dataset

class ClassificationDataset(Dataset):
    def __init__(self, file_paths, labels=None, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, index):
        file_path = self.file_paths[index]
        # image = Image.open(file_path).convert('RGB')
        image = Image.open(file_path).convert('L')

        if self.transform is not None:
            image = self.transform(image)

        if self.labels is not None:
            return image, self.labels[index]
        else:
            return image

## Model Design
The following section shows your **model structure.
You may modify any part—such as the number of layers, activation functions, or other design choices to improve the final performance.
However, please pay close attention to the ***total number of parameters*** in your model.


In [17]:
# model
import os
import numpy as np

import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image

class ClassificationModel(nn.Module):
    def __init__(self, Cin=1, C=16, Cout=100):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(Cin, C, 4, 2, 1, bias=False),
            nn.BatchNorm2d(C), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(C, C, 4, 2, 1, bias=False),
            nn.BatchNorm2d(C), nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(C, C, 4, 2, 1, bias=False),
            nn.BatchNorm2d(C),
            nn.ReLU(inplace=True),

            # nn.Flatten(),                               # (64)
            # nn.Linear(64, 100),                         # (100)
        )
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.drop = nn.Dropout(0.1)
        self.fc  = nn.Linear(C, Cout)


    def forward(self, x):
        # return self.main(x)

        x = self.features(x)         # 64→32→16→8→4→2 (for 64×64 input)
        x = self.gap(x).flatten(1)   # [N,C]
        x = self.drop(x)
        return self.fc(x)


## Check your Parameters
The following script can be used to compute the total number of parameters in your model.


In [18]:
model = ClassificationModel()
total_params = sum(p.numel() for p in model.parameters())
print("# parameters:", total_params)

# parameters: 10244


## Model Training
Ready to train!

This is only a baseline — you are free to explore different ideas.
Here are some possible directions:
- Adjusting the learning rate or using a scheduler
- Trying different optimizers (e.g., Adam, SGD, RMSprop)
- Adding data augmentations to make training more robust
- Using transfer learning with a pretrained model
- Redesigning or simplifying the model architecture

These are just examples. Be creative and explore what works best for you!


In [19]:
from tqdm.auto import tqdm

sports = sorted(os.listdir(TRAIN_DIR))
label_map = {sport:i for i, sport in enumerate(sports)}
num_classes = len(sports)


# Function to get file paths and labels for a given root directory
def get_file_paths_and_labels(root_dir, label_map):
    file_paths = []
    gt_sports = []
    valid_extensions = ['.jpg', '.jpeg']
    for sport in sorted(os.listdir(root_dir)):
        sport_root = os.path.join(root_dir, sport)
        if os.path.isdir(sport_root):
            for file_name in sorted(os.listdir(sport_root)):
                file_path = os.path.join(sport_root, file_name)
                if os.path.isfile(file_path) and any(file_path.lower().endswith(ext) for ext in valid_extensions): # Check if it's a file and has a valid extension
                    file_paths.append(file_path)
                    gt_sports.append(sport)
    labels = [label_map[sport] for sport in gt_sports]
    return file_paths, labels

train_file_paths, train_labels = get_file_paths_and_labels(TRAIN_DIR, label_map)
valid_file_paths, valid_labels = get_file_paths_and_labels(VALID_DIR, label_map)

# --- metrics helpers ---
@torch.no_grad()
def batch_topk_correct(logits, labels, k=5):
    topk = torch.topk(logits, k, dim=1).indices
    return topk.eq(labels.view(-1,1)).any(dim=1).float().sum().item()


#################
# MODIFY        #
#################

# --- transforms ---
train_tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])

valid_tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])

# --- datasets/loaders ---
train_dataset = ClassificationDataset(train_file_paths, train_labels, transform=train_tf)
valid_dataset = ClassificationDataset(valid_file_paths, valid_labels, transform=valid_tf)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=1, pin_memory=False)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False, num_workers=1, pin_memory=False)

# --- model / loss / optim ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ClassificationModel(Cin=1, Cout=num_classes).to(device)  # Cin=1 for grayscale
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


# --- train ---
num_epoch = 20
best_accuracy = 0.0
model_save_path = f"w_{STUDENT_ID}.pth"

for epoch in range(num_epoch):
    # --------------------- Train ---------------------
    model.train()
    train_loss, t1_correct, t5_correct, n = 0.0, 0, 0, 0
    pbar = tqdm(train_loader, ncols=120, desc=f"Train {epoch+1:02d}/{num_epoch}")
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        bs = labels.size(0)
        n += bs
        train_loss += loss.item() * bs
        t1_correct += (outputs.argmax(1) == labels).sum().item()
        t5_correct += batch_topk_correct(outputs, labels, k=5)

        pbar.set_postfix(
            loss=f"{train_loss/max(n,1):.4f}",
            top1=f"{(t1_correct/max(n,1))*100:.2f}%",
            top5=f"{(t5_correct/max(n,1))*100:.2f}%"
        )

    train_loss /= max(n, 1)
    train_top1 = t1_correct / max(n, 1) * 100
    train_top5 = t5_correct / max(n, 1) * 100

    # --------------------- Valid ---------------------
    model.eval()
    valid_loss, v1_correct, v5_correct, vn = 0.0, 0, 0, 0
    pbar = tqdm(valid_loader, ncols=120, desc="Valid")
    with torch.no_grad():
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            bs = labels.size(0)
            vn += bs
            valid_loss += loss.item() * bs
            v1_correct += (outputs.argmax(1) == labels).sum().item()
            v5_correct += batch_topk_correct(outputs, labels, k=5)

            pbar.set_postfix(
                loss=f"{valid_loss/max(vn,1):.4f}",
                top1=f"{(v1_correct/max(vn,1))*100:.2f}%",
                top5=f"{(v5_correct/max(vn,1))*100:.2f}%"
            )

    valid_loss /= max(vn, 1)
    valid_top1 = v1_correct / max(vn, 1) * 100
    valid_top5 = v5_correct / max(vn, 1) * 100

    # --------------------- Epoch print ---------------------
    print(f"Epoch [{epoch+1:02d}/{num_epoch}]")
    print(f"Train: loss {train_loss:.4f}, top1 {train_top1:.2f}%, top5 {train_top5:.2f}%")
    print(f"Valid: loss {valid_loss:.4f}, top1 {valid_top1:.2f}%, top5 {valid_top5:.2f}%")

    # save best by top-5
    if valid_top5 > best_accuracy:
        best_accuracy = valid_top5
        torch.save(model.state_dict(), model_save_path)

print(f"Best Validation Top-5: {best_accuracy:.2f}%  (saved to {model_save_path})")


Train 01/20:   0%|                                                                              | 0/106 [00:00…

Valid:   0%|                                                                                      | 0/4 [00:00…

Epoch [01/20]
Train: loss 4.5487, top1 2.96%, top5 10.29%
Valid: loss 4.4645, top1 3.60%, top5 13.20%


Train 02/20:   0%|                                                                              | 0/106 [00:00…

Valid:   0%|                                                                                      | 0/4 [00:00…

...

Valid:   0%|                                                                                      | 0/4 [00:00…

Epoch [19/20]
Train: loss 3.2518, top1 19.95%, top5 48.94%
Valid: loss 3.2354, top1 22.80%, top5 50.20%


Train 20/20:   0%|                                                                              | 0/106 [00:00…

Valid:   0%|                                                                                      | 0/4 [00:00…

Epoch [20/20]
Train: loss 3.2411, top1 20.33%, top5 48.93%
Valid: loss 3.2014, top1 22.60%, top5 51.00%
Best Validation Top-5: 53.00%  (saved to /content/drive/MyDrive/flattendata/w_313551121.pth)


## Model Evaluation
After completing training, you can use the script below to evaluate your model.
Run it to check the Top-1 and Top-5 accuracy on the test set.


In [20]:
import csv
import pandas as pd

WEIGHT_PATH = f"w_{STUDENT_ID}.pth"
OUT_CSV     = f"pred_{STUDENT_ID}.csv"

print("Using weights:", WEIGHT_PATH)

# ---------------- dataset ----------------
VALID_EXT = {".jpg", ".jpeg"}

class LoadFlatTest(Dataset):
    def __init__(self, root_dir, class_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        self.image_paths = [
            os.path.join(root_dir, fname)
            for fname in sorted(os.listdir(root_dir))
            if os.path.splitext(fname)[1].lower() in VALID_EXT
        ]

        self.classes = sorted([
            d for d in os.listdir(class_dir)
            if os.path.isdir(os.path.join(class_dir, d))
        ])
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        p = self.image_paths[idx]
        img = Image.open(p).convert("L")  # 1-channel grayscale
        if self.transform:
            img = self.transform(img)
        return img, 0

# ---------------- test routine ----------------
def test():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Testing on:", device)

    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])  # 1-channel stats
    ])

    dataset = LoadFlatTest(root_dir=TEST_DIR, class_dir=TRAIN_DIR, transform=transform)
    test_loader = DataLoader(dataset=dataset, batch_size=128, shuffle=False, num_workers=2, pin_memory=False)

    # IMPORTANT: Cout must match #classes
    num_classes = len(dataset.classes)

    # add args if your model needs
    model = ClassificationModel()

    # Load weights
    state = torch.load(WEIGHT_PATH, map_location="cpu")
    if isinstance(state, dict) and any(k in state for k in ("state_dict", "model_state_dict", "model")):
        for k in ("state_dict", "model_state_dict", "model"):
            if k in state:
                state = state[k]; break
    state = { (k.split('.',1)[1] if k.startswith("module.") else k): v for k,v in state.items() }
    model.load_state_dict(state, strict=False)

    model = model.to(device).eval()

    results, idx_start = [], 0
    with torch.no_grad():
        for images, _ in test_loader:
            images = images.to(device, non_blocking=True)
            logits = model(images)
            top5 = torch.topk(logits, k=min(5, num_classes), dim=1).indices.cpu()

            batch_paths = [dataset.image_paths[i] for i in range(idx_start, idx_start + images.size(0))]
            idx_start += images.size(0)

            for j, path in enumerate(batch_paths):
                fname = os.path.basename(path)
                idxs = top5[j].tolist()
                labels = [dataset.classes[i] for i in idxs]
                while len(labels) < 5:
                    labels.append(labels[-1] if labels else dataset.classes[0])
                results.append([fname] + labels[:5])

    df = pd.DataFrame(results, columns=["file_name", "pred1", "pred2", "pred3", "pred4", "pred5"])
    os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)
    df.to_csv(OUT_CSV, index=False)
    print("Saved predictions to:", OUT_CSV)

test()


Using weights: /content/drive/MyDrive/flattendata/w_313551121.pth
Testing on: cpu
Saved predictions to: /content/drive/MyDrive/flattendata/pred_313551121.csv


## Grading Criteria

### Top-5 Accuracy (75 points)
- **Accuracy ≥ 65%** → full points  
- **60% ≤ Accuracy < 65%** → 65 points  
- **55% ≤ Accuracy < 60%** → 55 points  
- **50% ≤ Accuracy < 55%** → 45 points  
- **Accuracy < 50%** → 0 points  

---

### Number of Parameters (15 points)
This evaluation is based on the number of parameters in your model.  
The fewer parameters you use, the higher score you will get.  

**Score formula:**  
\[
*${Round}\left( \frac{n-r}{n-1} \times 15 \right)$*
\]  

- n \: number of students  
- r \: your ranking (with \(r = 1\) being the best rank)  

---

### Report (10 points)
In the report, briefly describe your approach.  

For example:  
- The design of the model architecture  
- Strategies for balancing model performance with computational and parameter efficiency  
- The development of an effective training strategy  


